In [1]:
from docx import Document
import re
import os
# import openpyxl
from datetime import datetime
import pprint
import pandas as pd



transcripts_folder_path = os.getcwd()+'/raw_transcripts/'
time_format = "%H:%M:%S"
pretty_printer = pprint.PrettyPrinter(width=40, compact=True)
speaker_time_dict = {}

In [2]:
def match_pattern(pattern, text):
    # pattern = r"\['(?:\d+:)?\d+:\d+'\]"

    if re.search(pattern, text):
        return True
    else:
        return False

In [3]:
def find_doc_files_in_current_folder() -> list:
    """Find the docx file in the current folder
    It ignores the temperory files which are getting opened
    when files are opened '~'
    Returns:
        list: list of the files name as str
    """

    doc_files = []

    for filename in os.listdir(transcripts_folder_path):
        # filename.startswith("~") -> ignores the temprory files when that words make when file is open
        if filename.endswith(".docx") and not filename.startswith("~"):
            doc_files.append(filename)

    return doc_files

In [11]:
def calculate_time_delta(time1, time2):
    time1_parts = list(map(int, time1.split(':')))
    time2_parts = list(map(int, time2.split(':')))

    while len(time1_parts) < 3:
        time1_parts.insert(0, 0)
    while len(time2_parts) < 3:
        time2_parts.insert(0, 0)

    hours_diff = time2_parts[0] - time1_parts[0]
    minutes_diff = time2_parts[1] - time1_parts[1]
    seconds_diff = time2_parts[2] - time1_parts[2]

    total_seconds = hours_diff * 3600 + minutes_diff * 60 + seconds_diff
    return total_seconds

In [4]:
list_file_names = find_doc_files_in_current_folder()
list_file_names


doc = Document('test close caption.docx')

# dict_item ={
#     1 :{
#         'start_time': 000, 
#         'character_lenght': 123123, #joint list then find the lenght of string
#         'text': ['']
#     },
#     2 :{

#     },
# }

dict_item ={}

discussion_number = 1 #! per file
file_speech_per_person = []
tmp_per_person_per_speech = []
rgl_time_pt = r'\b\d+:\d+\s'
ignore_header_count = 0

for paragraph in doc.paragraphs:
    text = paragraph.text
    
    text_detail = text.split(">>")
    
    if text_detail != ['']:

        if ignore_header_count < 2:
            #! this will help to get ride of the title and youtube linke
            ignore_header_count += 1

        elif len(text_detail) == 1 and match_pattern(rgl_time_pt, text=text_detail[0]):
            #! this will find the latest time stamp in the file
            temp_time = text_detail[0]
            print(temp_time)

        elif len(text_detail) == 1 :
            tmp_per_person_per_speech.append(text_detail[0])

        elif len(text_detail) == 2 and  text_detail[0] == '':
            if tmp_per_person_per_speech != []:
                
                file_speech_per_person.append(tmp_per_person_per_speech)
            
            tmp_per_person_per_speech=[]
            tmp_per_person_per_speech.append(text_detail[1])

        elif len(text_detail) == 2:
            tmp_per_person_per_speech.append(text_detail[0])
            file_speech_per_person.append(tmp_per_person_per_speech)
            tmp_per_person_per_speech=[]
            tmp_per_person_per_speech.append(text_detail[1])

        elif len(text_detail) > 2:
            
            for indx in range(len(text_detail)):

                if indx == 0 and text_detail[indx] == ['']:
                    file_speech_per_person.append(tmp_per_person_per_speech)
                    tmp_per_person_per_speech=[]

                elif len(text_detail) == indx+1 :
                    tmp_per_person_per_speech.append(text_detail[indx])

                else:
                    tmp_per_person_per_speech.append(text_detail[indx])
                    file_speech_per_person.append(tmp_per_person_per_speech)
                    tmp_per_person_per_speech=[]

file_speech_per_person.append(tmp_per_person_per_speech)


# for item in file_speech_per_person:
#     print(item)


0:00 
0:05 
0:11 
0:17 
0:24 
0:32 
0:37 
0:43 
0:50 
0:55 
1:00 
1:05 
1:11 
1:18 
1:25 
1:33 
1:39 
1:44 
1:51 
1:57 
2:03 
2:09 
2:14 
2:20 
2:28 
2:33 
2:41 
2:46 
2:52 
2:57 
3:07 
3:14 
3:19 
3:25 
3:30 
3:35 
3:40 
3:46 
3:55 
4:01 
4:09 
4:15 
4:21 
4:27 
4:34 
4:39 
4:45 
4:54 
5:02 
5:07 
5:15 
5:23 
5:30 
5:37 
5:44 
5:50 
5:58 
6:05 
6:12 
6:18 
6:24 
6:30 
6:38 
6:44 
6:50 
6:57 
7:07 
7:14 
7:20 
7:25 
7:31 
7:36 
7:42 
7:47 
7:53 
8:00 
8:07 
8:12 
8:19 
8:27 
8:35 
8:43 
8:50 
9:00 
9:06 
9:13 
9:21 
9:29 
9:35 
9:43 
9:51 
9:58 
10:04 
10:10 
10:15 
10:20 
10:26 
10:33 
10:39 
10:46 
10:52 
11:00 
11:07 
11:14 
11:22 
11:30 
11:37 
11:45 
11:51 
11:57 
12:07 
12:12 
12:17 
12:24 
12:31 
12:39 
12:45 
12:51 
12:56 
13:04 
13:13 
13:21 
13:30 
13:36 
13:41 
13:48 
13:54 
14:02 
14:07 
14:14 
14:21 
14:27 
14:33 
14:38 
14:47 
14:53 
15:00 
15:05 
15:13 
15:19 
15:28 
15:35 
15:43 
15:51 
15:56 
16:02 
16:11 
16:16 
16:23 
16:31 
16:38 
16:45 
16:51 
17:03 
17:09 
17:17 


In [5]:
rgl_time_pt = "\['(?:\d+:)?\d+:\d+'\]"

In [10]:
def calculate_time_delta(time1, time2):
    time1_parts = list(map(int, time1.split(':')))
    time2_parts = list(map(int, time2.split(':')))

    while len(time1_parts) < 3:
        time1_parts.insert(0, 0)
    while len(time2_parts) < 3:
        time2_parts.insert(0, 0)

    hours_diff = time2_parts[0] - time1_parts[0]
    minutes_diff = time2_parts[1] - time1_parts[1]
    seconds_diff = time2_parts[2] - time1_parts[2]

    total_seconds = hours_diff * 3600 + minutes_diff * 60 + seconds_diff
    return total_seconds

time1 = '00:10'
time2 = '2:00:00'

result = calculate_time_delta(time1, time2)
print("Delta:", result)  # Output: Delta: 1547


Delta: 7190
